In [1]:
import pandas as pd
import numpy as np

In [2]:
#import relevant data
master = pd.read_csv('data/master.csv')
citations = pd.read_csv('data/citations.csv', dtype={'patent_number':str})
focal_citation_link = pd.read_csv('data/citations_patents_level.csv')
focal_class = pd.read_csv('data/uspcs_level.csv')

In [3]:
master.head()

,patent_number,grant_year,app_year,num_inventors,us_inventor,cite_foreign_patent,is_missing,num_design_cited,num_utility_cited,non-pat_refs,num_figures,num_assignees,priority_date
0,D257752,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1,1980
1,D257924,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1,1980
2,D258382,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1,1980
3,D258383,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1,1980
4,D258571,1981,1980,1,1.0,0.0,1,1,1,0.0,2.0,1,1980


In [4]:
focal = master[['patent_number','priority_date']]
focal.head()

,patent_number,priority_date
0,D257752,1980
1,D257924,1980
2,D258382,1980
3,D258383,1980
4,D258571,1980


In [5]:
citations.head()

,uspc_mainclass_id,uspc_subclass_id,patent_number
0,248,248/423,3930272
1,403,403/107,3930272
2,5,5/11,3930272
3,5,5/100,3930272
4,47,47/58.1R,3930335


In [6]:
focal_citation_link.head()

,cited_patent_number,patent_number
0,4162014,D257752
1,4162014,D257924
2,D253842,D258382
3,D253842,D258383
4,D253842,D258678


In [7]:
focal_citation_link.isnull().sum()

cited_patent_number    0
patent_number          0
dtype: int64

In [8]:
focal_citation = pd.merge(focal, focal_citation_link, on='patent_number', how='left')
focal_citation.head()

,patent_number,priority_date,cited_patent_number
0,D257752,1980,4162014
1,D257924,1980,4162014
2,D258382,1980,D253842
3,D258383,1980,D253842
4,D258571,1980,NaN


In [9]:
focal_citation['patent_number'].nunique()

525490

In [10]:
focal_citation.isnull().sum()

patent_number              0
priority_date              0
cited_patent_number    18363
dtype: int64

Let's see what is null and why

In [11]:
focal_citation.loc[focal_citation['cited_patent_number'].isnull()]

,patent_number,priority_date,cited_patent_number
4,D258571,1980,NaN
5,D258650,1980,NaN
8,D258760,1980,NaN
21,D259183,1980,NaN
45,D259575,1980,NaN
46,D259610,1980,NaN
56,D259669,1980,NaN
63,D259753,1980,NaN
65,D259820,1980,NaN
75,D259959,1980,NaN


After viewing a sample, these patents either don't make references, rare, or all of their references are pre 1976, more common. Since they are not useful for analysis, I will drop them

In [12]:
# percentage of patents affected
focal_citation['cited_patent_number'].value_counts(normalize=True, dropna=False).head()

NaN        0.002822
D504889    0.000077
D337569    0.000071
D341848    0.000068
D561358    0.000059
Name: cited_patent_number, dtype: float64

Less than 1 percent of patents are affected by this

In [13]:
focal_citation.dropna(inplace=True)
focal_citation.patent_number.nunique()

507127

# current number of patents represened: 507,127

In [14]:
citations.rename(index=str, columns={'patent_number':'cited_patent_number'}, inplace=True)
focal_citation_citeclass = pd.merge(focal_citation, citations, on='cited_patent_number', how='left')
focal_citation_citeclass.patent_number.nunique()

507127

In [15]:
focal_citation_citeclass.head()

,patent_number,priority_date,cited_patent_number,uspc_mainclass_id,uspc_subclass_id
0,D257752,1980,4162014,211,211/50
1,D257752,1980,4162014,211,211/55
2,D257752,1980,4162014,211,211/126.1
3,D257924,1980,4162014,211,211/50
4,D257924,1980,4162014,211,211/55


In [16]:
focal_citation_citeclass.rename(index=str, columns={'uspc_mainclass_id':'cite_mainclass', 'uspc_subclass_id':'cite_subclass'}, inplace=True)
focal_citation_citeclass.isnull().sum()

patent_number            0
priority_date            0
cited_patent_number      0
cite_mainclass         530
cite_subclass          530
dtype: int64

Drop nulls. Without classification its useles

In [19]:
focal_citation_citeclass.dropna(inplace=True)
focal_citation_citeclass['patent_number'].nunique()

507126

In [21]:
focal_citation_citeclass.loc[(focal_citation_citeclass.cite_mainclass.str.contains('D')) & (~focal_citation_citeclass.cited_patent_number.str.contains('D'))]

,patent_number,priority_date,cited_patent_number,cite_mainclass,cite_subclass
36,D258766,1980,4069641,D25,D25/48.7
37,D258766,1980,4069641,D25,D25/119
54,D258766,1980,4184297,D25,D25/56
55,D258766,1980,4184297,D25,D25/123
90,D259281,1980,4098036,D25,D25/1
102,D259509,1980,3935984,D24,D24/165
116,D259510,1980,3952338,D25,D25/2
154,D259740,1980,4098036,D25,D25/1
215,D260549,1980,4164826,D22,D22/126
222,D260724,1980,3942591,D08,D08/7


In [22]:
citations.head()

,uspc_mainclass_id,uspc_subclass_id,cited_patent_number
0,248,248/423,3930272
1,403,403/107,3930272
2,5,5/11,3930272
3,5,5/100,3930272
4,47,47/58.1R,3930335


In [23]:
citations.loc[citations.cited_patent_number.str.match('4069641')]

,uspc_mainclass_id,uspc_subclass_id,cited_patent_number
56717,52,52/202,4069641
56718,52,52/476,4069641
56719,52,52/717.1,4069641
56720,D25,D25/48.7,4069641
56721,D25,D25/119,4069641


# Current number of patents represented: 507,126

In [ ]:
final = pd.merge(focal_citation_citeclass, focal_class, on='patent_number', how='left')
final.head()

In [ ]:
final.rename(index=str, columns={'uspc_mainclass_id':'focal_mainclass', 'uspc_subclass_id':'focal_subclass'}, inplace=True)

In [ ]:
final.shape

In [ ]:
final.patent_number.nunique()

In [ ]:
final.isnull().sum()

4000 of our patents had null citations. These are mainly withdrawn patents and newly granted patent. I assume this has to do with patentsview getting updated information

In [ ]:
final.dropna(inplace=True)
final['patent_number'].nunique()

# Current patent count: 503,128

In [ ]:
final.to_csv('data/final_no_drops.csv', index_label=False)

In [2]:
final = pd.read_csv('data/final_no_drops.csv')
final.head()

,patent_number,priority_date,cited_patent_number,cite_mainclass,cite_subclass,focal_mainclass,focal_subclass
0,D257752,1980,4162014,211,211/50,D19,D19/75
1,D257752,1980,4162014,211,211/55,D19,D19/75
2,D257752,1980,4162014,211,211/126.1,D19,D19/75
3,D257924,1980,4162014,211,211/50,D06,D06/573
4,D257924,1980,4162014,211,211/55,D06,D06/573


In [6]:
final.focal_mainclass.sort_values().unique()

array(['1', '100', '101', '102', '104', '105', '106', '108', '111', '114',
       '116', '118', '119', '12', '123', '124', '125', '126', '128',
       '132', '135', '137', '138', '14', '141', '142', '144', '15', '150',
       '157', '16', '160', '165', '168', '169', '172', '173', '174',
       '175', '177', '180', '181', '182', '184', '188', '19', '190',
       '194', '198', '2', '200', '206', '210', '211', '215', '216', '217',
       '219', '220', '221', '222', '223', '224', '227', '229', '232',
       '235', '239', '24', '241', '242', '244', '248', '249', '251',
       '254', '256', '257', '26', '261', '267', '269', '27', '271', '273',
       '277', '279', '280', '281', '283', '285', '289', '292', '293',
       '294', '296', '297', '30', '301', '305', '312', '313', '323', '33',
       '34', '340', '341', '342', '343', '345', '346', '347', '348',
       '351', '353', '356', '358', '359', '36', '360', '361', '362',
       '366', '368', '374', '378', '379', '381', '382', '383', '384',
 

In [30]:
final.cite_mainclass.sort_values().unique()

array(['1', '100', '101', '102', '104', '105', '106', '108', '109', '110',
       '111', '112', '114', '116', '117', '118', '119', '12', '122',
       '123', '124', '125', '126', '127', '128', '131', '132', '134',
       '135', '136', '137', '138', '139', '14', '140', '141', '142',
       '144', '147', '148', '149', '15', '150', '152', '156', '157',
       '159', '16', '160', '162', '163', '164', '165', '166', '168',
       '169', '171', '172', '173', '174', '175', '177', '178', '180',
       '181', '182', '184', '185', '186', '187', '188', '19', '190',
       '191', '192', '193', '194', '196', '198', '199', '2', '200', '201',
       '202', '203', '204', '205', '206', '208', '209', '210', '211',
       '212', '213', '215', '216', '217', '218', '219', '220', '221',
       '222', '223', '224', '225', '226', '227', '228', '229', '23',
       '231', '232', '234', '235', '236', '237', '238', '239', '24',
       '241', '242', '244', '245', '246', '248', '249', '250', '251',
       '252', '25

There are inconsitency in some of the class name. Ex: D01 and D1. This needs to be reconciled

In [8]:
class_dictionary = {'D1':'D01', 'D2':'D02', 'D3':'D03', 'D4':'D04', 'D5':'D05', 'D6':'D06', 'D7':'D07', 'D8':'D08', 'D9':'D09'}

In [55]:
final['cite_mainclass'] = final['cite_mainclass'].replace(class_dictionary)
final['focal_mainclass'] = final['focal_mainclass'].replace(class_dictionary)

In [57]:
final.cite_mainclass.sort_values().unique()

array(['1', '100', '101', '102', '104', '105', '106', '108', '109', '110',
       '111', '112', '114', '116', '117', '118', '119', '12', '122',
       '123', '124', '125', '126', '127', '128', '131', '132', '134',
       '135', '136', '137', '138', '139', '14', '140', '141', '142',
       '144', '147', '148', '149', '15', '150', '152', '156', '157',
       '159', '16', '160', '162', '163', '164', '165', '166', '168',
       '169', '171', '172', '173', '174', '175', '177', '178', '180',
       '181', '182', '184', '185', '186', '187', '188', '19', '190',
       '191', '192', '193', '194', '196', '198', '199', '2', '200', '201',
       '202', '203', '204', '205', '206', '208', '209', '210', '211',
       '212', '213', '215', '216', '217', '218', '219', '220', '221',
       '222', '223', '224', '225', '226', '227', '228', '229', '23',
       '231', '232', '234', '235', '236', '237', '238', '239', '24',
       '241', '242', '244', '245', '246', '248', '249', '250', '251',
       '252', '25

No all patents have a typical class assignements or citations. Going to remove them from the master dataset and put them aside for further analysis

In [58]:
# remove cite_mainclass from no longer published, or plant

In [70]:
special = final.loc[(final.cite_mainclass.str.contains('No longer published')) | (final.cite_mainclass.str.contains('PLT'))]

In [72]:
final = final.drop(special.index)

In [73]:
final.patent_number.nunique()

503117

# Current patent count: 503,117

In [81]:
# remove design patent with utility class and utility patent with design class
design_with_util_class = final.loc[(~final.focal_mainclass.str.contains('D'))]

In [82]:
final = final.drop(design_with_util_class.index)

In [83]:
utility_with_design_class = final.loc[(~final.cited_patent_number.str.contains('D')) & (final.cite_mainclass.str.contains('D'))]
final = final.drop(utility_with_design_class.index)

In [84]:
final.patent_number.nunique()

503107

# Current patent count: 503,107

This is with very tradional design and utility classes

In [86]:
outcast = pd.concat([special, design_with_util_class, utility_with_design_class])

In [90]:
outcast.patent_number.nunique()

102972

102,972 patents are affected by these extraordinary catogories, but only 21 patents overall were unusable

In [91]:
#save
final.to_csv('data/final.csv', index_label=False)
outcast.to_csv('data/outcast.csv', index_label=False)
